In [42]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# 1. Connect to Qdrant
client = QdrantClient("http://127.0.0.1:6333")


In [2]:
from config import EMBEDDING_MODEL_PATH
from embedding import OllamaEmbedding
import torch

model = OllamaEmbedding()
model.load_state_dict(torch.load(EMBEDDING_MODEL_PATH / "state_dict.pt"))

<All keys matched successfully>

In [43]:
assert client.create_collection(
    collection_name="keywords",
    vectors_config=VectorParams(size=model.embed_dim, distance=Distance.COSINE),
    on_disk_payload=False,
)


In [44]:
import tqdm.auto as tqdm

points = list(model.embed_keys.items())


batch_size = 128
batches = tqdm.trange(0, len(points), batch_size)

for i in batches:
    vectors = model.embed_cache[i : i + batch_size].tolist()
    batch = [
        {"id": i, "vector": vectors[b_i], "payload": {"keyword": text}}
        for b_i, (text, i) in enumerate(points[i : i + batch_size])
    ]

    client.upsert(collection_name="fa_keywords_custom", points=batch)


  0%|          | 0/452 [00:00<?, ?it/s]

In [70]:
%%time

query = "1020000000"
query_vec = model.embed([query])[0].tolist()


CPU times: total: 0 ns
Wall time: 1.01 ms


In [ ]:
%%time

results = client.query_points(
    collection_name="fa_keywords",
    query=query_vec,
    limit=10,
)

related_keywords = []
for r in results.points:
    related_keywords.append(r.payload["keyword"])
    print(r.payload["keyword"], "→", r.score)


1020000000 → 1.0
آیفون 100000000000000. → 0.6965221
996339 → 0.64913654
قیمت 0.00001 بیت کوین به تومان → 0.63823617
10000 لیره به افغانی → 0.6199794
1000 دالر به افغانی → 0.61823577
1000 درهم به افغانی → 0.6172321
1000 لیره به افغانی → 0.6163437
10000 دقیقه مکالمه رایگان رایتل → 0.6160077
100 انیمیشن برتر دنیا → 0.6150603
CPU times: total: 0 ns
Wall time: 27.5 ms
